In [1]:
import pandas as pd
import numpy as np
import re

import json

from IPython.display import JSON



In [8]:
data = pd.read_csv('chicago-food-inspections/food-inspections.csv')

In [4]:
data.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2320315,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,2216009.0,Daycare Above and Under 2 Years,Risk 1 (High),1300 W 99TH ST,CHICAGO,IL,60643.0,...,Pass,NaN,41.714168,-87.655291,"{'longitude': '41.7141680989703', 'latitude': ...",NaN,NaN,NaN,NaN,NaN
1,2320342,YOLK TEST KITCHEN,YOLK TEST KITCHEN,2589655.0,Restaurant,Risk 1 (High),1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,...,Pass w/ Conditions,23. PROPER DATE MARKING AND DISPOSITION - Comm...,41.913588,-87.682203,"{'longitude': '41.9135877900482', 'latitude': ...",NaN,NaN,NaN,NaN,NaN
2,2320328,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,2583309.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,...,Out of Business,NaN,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':...",NaN,NaN,NaN,NaN,NaN
3,2320319,LA PALAPITA,LA PALAPITA,2694702.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,...,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':...",NaN,NaN,NaN,NaN,NaN
4,2320228,47TH ST CANTINA,47TH ST CANTINA,2678250.0,Liquor,Risk 3 (Low),4311 W 47TH ST,CHICAGO,IL,60632.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.807662,-87.731480,"{'longitude': '41.80766199360051', 'latitude':...",NaN,NaN,NaN,NaN,NaN


In [30]:
data['DBA Name'].nunique()

27476

In [252]:
display(data)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2320315,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,2216009.0,Daycare Above and Under 2 Years,Risk 1 (High),1300 W 99TH ST,CHICAGO,IL,60643.0,...,Pass,NaN,41.714168,-87.655291,"{'longitude': '41.7141680989703', 'latitude': ...",NaN,NaN,NaN,NaN,NaN
1,2320342,YOLK TEST KITCHEN,YOLK TEST KITCHEN,2589655.0,Restaurant,Risk 1 (High),1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,...,Pass w/ Conditions,23. PROPER DATE MARKING AND DISPOSITION - Comm...,41.913588,-87.682203,"{'longitude': '41.9135877900482', 'latitude': ...",NaN,NaN,NaN,NaN,NaN
2,2320328,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,2583309.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,...,Out of Business,NaN,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':...",NaN,NaN,NaN,NaN,NaN
3,2320319,LA PALAPITA,LA PALAPITA,2694702.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,...,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':...",NaN,NaN,NaN,NaN,NaN
4,2320228,47TH ST CANTINA,47TH ST CANTINA,2678250.0,Liquor,Risk 3 (Low),4311 W 47TH ST,CHICAGO,IL,60632.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.807662,-87.731480,"{'longitude': '41.80766199360051', 'latitude':...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194809,52234,Cafe 608,Cafe 608,2013328.0,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657.0,...,Pass,NaN,41.938007,-87.644755,"{'longitude': '41.938006880423615', 'latitude'...",NaN,NaN,NaN,NaN,NaN
194810,67738,MICHAEL'S ON MAIN CAFE,MICHAEL'S ON MAIN CAFE,2008948.0,Restaurant,Risk 1 (High),8750 W BRYN WAWR AVE,CHICAGO,IL,60631.0,...,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194811,67733,WOLCOTT'S,TROQUET,1992040.0,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613.0,...,Pass,NaN,41.961606,-87.675967,"{'longitude': '41.961605669949854', 'latitude'...",NaN,NaN,NaN,NaN,NaN
194812,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279.0,Restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601.0,...,Pass,NaN,41.884586,-87.631010,"{'longitude': '41.88458626715456', 'latitude':...",NaN,NaN,NaN,NaN,NaN


## Yelp Database

In [2]:
import os
from yelpapi import YelpAPI

YELP_API_KEY = os.environ['YELP_API_KEY']
YELP_CLIENT_ID = os.environ['YELP_CLIENT_ID']
client = YelpAPI(YELP_API_KEY)

In [41]:
# The yelp_df is a selection of columns of the original df, containing only location information
# We then drop all duplicates and reset the index in order to more easily cross reference with the new data
# We also include a id col corresponding to the yelp id, and a count of how many results returned
yelp_df = pd.read_pickle('yelp/yelp.pkl')

# Contains the scrapped data from yelp
scrapped = pd.read_pickle('yelp/scrapped.pkl')


#
business = pd.read_pickle('yelp/business.pkl')
reviews = pd.read_pickle('yelp/reviews.pkl')

In [4]:
def business_match_to_df_row(business_):
    business = business_.copy()
    business.update(business.pop('coordinates'))
    business.update(business.pop('location'))
    business.pop('phone')
    business.pop('display_phone')
    business.pop('country')
    if str(business['address2']) == '' or not business['address2']:
        del business['address2']
    if str(business['address3']) == '' or not business['address3']:
        business.pop('address3', None)
    return business

In [59]:

for i in range(1500,4500):
    # Get the i-th row of the data frame
    row = yelp_df.iloc[i]
    # Make sure we don't request data we already have
    if row['matches'] == -1:
        query = client.business_match_query(name=row['AKA Name'],
                                            city=row['City'],
                                            state=row['State'],
                                            country='US',
                                            address1=row['Address'])
        # We note how many matches we got and only take the first one
        yelp_df.loc[yelp_df.index[i], 'matches'] = len(query['businesses'])
        if len(query['businesses']) == 1:
            yelp_df.loc[yelp_df.index[i], 'id'] = query['businesses'][0]['id']
            scrapped = scrapped.append(pd.DataFrame([business_match_to_df_row(query['businesses'][0])]))

In [71]:
df = yelp_df[yelp_df['id'].notnull()].merge(scrapped,on='id')
df

,index,DBA Name,AKA Name,Address,City,State,Zip,Latitude,Longitude,Zip Codes,...,address2,address3,alias,city,display_address,latitude,longitude,name,state,zip_code
0,0,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,1300 W 99TH ST,CHICAGO,IL,60643.0,41.714168,-87.655291,NaN,...,NaN,NaN,serendipity-childcare-chicago,Chicago,"[1300 W 99th St, Chicago, IL 60643]",41.714330,-87.655420,Serendipity Childcare,IL,60643
1,1,YOLK TEST KITCHEN,YOLK TEST KITCHEN,1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,41.913588,-87.682203,NaN,...,NaN,NaN,yolk-test-kitchen-bucktown-chicago,Chicago,"[1767 N Milwaukee Ave, Chicago, IL 60647]",41.913692,-87.682058,Yolk Test Kitchen - Bucktown,IL,60647
2,2,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,3834 W 47TH ST,CHICAGO,IL,60632.0,41.808025,-87.720037,NaN,...,NaN,NaN,las-asadas-chicago,Chicago,"[3834 W 47th St, Chicago, IL 60632]",41.808170,-87.720110,Las Asadas,IL,60632
3,5,CRAZY BIRD,CRAZY BIRD,1160 W GRAND AVE,CHICAGO,IL,60642.0,41.891193,-87.657055,NaN,...,NaN,NaN,crazy-bird-chicago-2,Chicago,"[1160 W Grand Ave, Chicago, IL 60642]",41.891326,-87.656985,Crazy Bird,IL,60642
4,6,THE REDHEAD PIANO BAR,THE REDHEAD PIANO BAR,16-18 W ONTARIO ST,CHICAGO,IL,60654.0,41.893371,-87.628783,NaN,...,NaN,NaN,the-redhead-piano-bar-chicago,Chicago,"[16 W Ontario St, Chicago, IL 60654]",41.893530,-87.628740,Redhead Piano Bar,IL,60654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,6658,SLICE FACTORY,SLICE FACTORY,3435 1/2 W 26TH ST,CHICAGO,IL,60623.0,41.844322,-87.711274,NaN,...,NaN,NaN,slice-factory-chicago-2,Chicago,"[3435 1/2 W 26th S, Chicago, IL 60623]",41.844270,-87.711460,Slice Factory,IL,60623
3372,6660,Great Sea Chinese Restaurant,Great Sea Chinese Restaurant,3254 W LAWRENCE AVE,CHICAGO,IL,60625.0,41.968541,-87.710704,NaN,...,NaN,NaN,great-sea-restaurant-chicago,Chicago,"[3253 W Lawrence Ave, Chicago, IL 60625]",41.968311,-87.710637,Great Sea Restaurant,IL,60625
3373,6662,ABRAM GALE,ABRAM GALE,2366-2370 N NEVA AVE,CHICAGO,IL,60707.0,41.923010,-87.804536,NaN,...,NaN,NaN,abram-gale-chicago,Chicago,"[2366 N Neva Ave, Chicago, IL 60707]",41.922671,-87.804447,Abram Gale,IL,60707
3374,6663,HAMPTON INN & SUITES,HAMPTON INN & SUITES,33 W ILLINOIS ST,CHICAGO,IL,60654.0,41.890729,-87.629386,NaN,...,NaN,NaN,hampton-inn-and-suites-chicago-downtown-chicago,Chicago,"[33 W Illinois, Chicago, IL 60654]",41.890681,-87.629299,Hampton Inn & Suites Chicago-Downtown,IL,60654


In [107]:
yelp_df.to_pickle('yelp/yelp.pkl')
scrapped.to_pickle('yelp/scrapped.pkl')
business.to_pickle('yelp/business.pkl')
reviews.to_pickle('yelp/reviews.pkl')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [89]:
# Get all other information from business
wanted_business = ['id','is_closed','review_count','categories','rating','price','transactions']
for i in range(10):
    row = scrapped.iloc[i]
    if not business['id'].isin([row['id']]).any():
        query = client.business_query(id=row['id'])
        result = dict((k, query[k]) for k in wanted_business if k in query)
        business = business.append([result])
        
        

7xiYs3CWFbs9lnqor7iRpQ
KPblL_UbpgEwUbZEUscTvA
GkW4K_g-wtNCHuF0w8LMYA
yz-3hfOJYKRnPRBOFPpHqQ
2D4YeUTn2GGtrcLoHSpvJQ
Q-0hZSru1U4xSXoM6MdjtA
Qk7K6Y1S6rmduUNch70bsA
Q-0hZSru1U4xSXoM6MdjtA
dv8o6WDhQB9SkQiS0kytrQ
TZv2SNMJ7jnF1dcre3bLPw


In [99]:
wanted_reviews = ['id','text','rating','time_created']
for i in range(10):
    row = scrapped.iloc[i]
    if not reviews['id'].isin([row['id']]).any():
        for review in query['reviews']:
            query = client.reviews_query(id=row['id'])
            result = dict((k, review[k]) for k in wanted_reviews if k in review)
            result.update({'business_id':row['id']})
            reviews = reviews.append([result])

In [101]:
# Compare the original data with scraped and combine with more business details
yelp_df[yelp_df['id'].notnull()].merge(scrapped,on='id').merge(business,on='id')

,index,DBA Name,AKA Name,Address,City,State,Zip,Latitude,Longitude,Zip Codes,...,longitude,name,state,zip_code,is_closed,review_count,categories,rating,price,transactions
0,5,CRAZY BIRD,CRAZY BIRD,1160 W GRAND AVE,CHICAGO,IL,60642.0,41.891193,-87.657055,NaN,...,-87.656985,Crazy Bird,IL,60642,False,75,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.5,$,"[pickup, delivery]"
1,6,THE REDHEAD PIANO BAR,THE REDHEAD PIANO BAR,16-18 W ONTARIO ST,CHICAGO,IL,60654.0,41.893371,-87.628783,NaN,...,-87.628740,Redhead Piano Bar,IL,60654,False,529,"[{'alias': 'musicvenues', 'title': 'Music Venu...",3.0,$$,[]
2,7,HAISOUS,CA PHE DA,1800-1802 S CARPENTER ST,CHICAGO,IL,60608.0,41.857963,-87.653358,NaN,...,-87.653500,Ca Phe Da Vietnamese Cafe,IL,60608,False,144,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,$$,"[pickup, delivery]"
3,8,HARMONY RESTAURANT,HARMONY RESTAURANT,6525 W ARCHER AVE,CHICAGO,IL,60638.0,41.792164,-87.785406,NaN,...,-87.785297,Harmony Restaurant,IL,60638,False,129,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.0,$$,"[pickup, delivery]"
4,10,HIMALAYAN SHERPA KITCHEN,HIMALAYAN SHERPA KITCHEN,2701 W LAWRENCE AVE,CHICAGO,IL,60625.0,41.968442,-87.696304,NaN,...,-87.696393,Himalayan Sherpa Kitchen,IL,60625,False,23,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",4.5,NaN,"[delivery, pickup]"
5,11,7 ELEVEN #35750H,7 ELEVEN,1658 N MILWAUKEE AVE,CHICAGO,IL,60647.0,41.911443,-87.679334,NaN,...,-87.679586,7-Eleven,IL,60647,False,5,"[{'alias': 'convenience', 'title': 'Convenienc...",3.0,$,[]
6,11,7 ELEVEN #35750H,7 ELEVEN,1658 N MILWAUKEE AVE,CHICAGO,IL,60647.0,41.911443,-87.679334,NaN,...,-87.679586,7-Eleven,IL,60647,False,5,"[{'alias': 'convenience', 'title': 'Convenienc...",3.0,$,[]
7,13,7-ELEVEN #35750A,7-ELEVEN,1658 N MILWAUKEE AVE,CHICAGO,IL,60647.0,41.911443,-87.679334,NaN,...,-87.679586,7-Eleven,IL,60647,False,5,"[{'alias': 'convenience', 'title': 'Convenienc...",3.0,$,[]
8,13,7-ELEVEN #35750A,7-ELEVEN,1658 N MILWAUKEE AVE,CHICAGO,IL,60647.0,41.911443,-87.679334,NaN,...,-87.679586,7-Eleven,IL,60647,False,5,"[{'alias': 'convenience', 'title': 'Convenienc...",3.0,$,[]
9,12,WALGREENS #10771,WALGREENS #10771,6460 W FULLERTON AVE,CHICAGO,IL,60707.0,41.923769,-87.787626,NaN,...,-87.788128,Walgreens,IL,60707,True,4,"[{'alias': 'drugstores', 'title': 'Drugstores'...",2.5,$$,[]
